In [41]:
# Melanie Schwartz
# sno122

## Lab 7A

<em>Lab 7 consists of two exercises on relational and document-oriented databases.</em>

In this exercise, you will read in an sqlite database ```sf_food_program_db.sqlite``` that contains data on San Francisco restaurants and health inspections. Be sure this database file is included in the same directory as this notebook. This data comes from [PADJO at Stanford University](http://2015.padjo.org/briefs/data_notes/sql-explore-sf-restaurant-data/). This data has three tables:

* ```businesses``` - data about each restaurant
* ```inspections``` - data about every health inspection made at a restaurant
* ```violations``` - data about every violation that a restaurant has raised

Familiarize yourself with the database by printing out the tables. The common column across the three tables is ```business_id```. You will run a series of SQL statements to answer some questions. 

**Tasks**

The code to connect to the sqlite database has already been written for you. Complete the following tasks.

1. Identify the top 10 restaurants in San Francisco have the most number of health code violations recorded in this database. Run a SQL query using ```read_sql()``` to get the results in a ```DataFrame```. The result should include the business_id, name, address, latitude, longitude, and count of the number of violations. What were the nature of those violations?

2. Create a Folium map with markers for the top 50 restaurants in San Francisco have the most number of health code violations recorded in this database. For the markers, include the name, the address, and the number of health code violations. Are many of these restaurants clustered together in geographically? If so, where in town do you see several restaurants with high numbers of health code violations?

In [43]:
import sqlite3
import pandas as pd
import numpy as np
import folium

# The sample data is provided as an sqlite database file. This code will open the file and you will be able
# start querying the database immediately.
connection = sqlite3.connect( 'sf_food_program_db.sqlite' )

# TODO: 1. Identify the top 10 restaurants in San Francisco have the most number of health code violations recorded in this database. Run a SQL query using 'read_sql()' to get the results in a 'DataFrame'. The result should include the business_id, name, address, latitude, longitude, and count of the number of violations. What were the nature of those violations?

# Enter your query here to identify the top 10 restaurants in San Francisco with the most number of violations

# In comments here, describe the nature of those violations

# Query to get the top 10 restaurants with the most violations
query10 = """
    SELECT b.business_id, b.name, b.address, b.latitude, b.longitude, COUNT(*) AS violation_count
    FROM businesses b
    JOIN violations v ON b.business_id = v.business_id
    GROUP BY b.business_id
    ORDER BY violation_count DESC
    LIMIT 10
    """

# Save query to DF
top_10 = pd.read_sql(query10, connection)

# clean data with missing lon or lat 
cleaned_top_10 = top_10.dropna(subset=['latitude', 'longitude'])

# print top 10 restaurants with violations
print(cleaned_top_10)

# TODO: 2. Create a Folium map with markers for the top 50 restaurants in San Francisco have the most number of health code violations recorded in this database. For the markers, include the name, the address, and the number of health code violations. Are many of these restaurants clustered together in geographically? If so, where in town do you see several restaurants with high numbers of health code violations?

# Create your map here - This has been started for you.

# In comments here, respond to the question on if there seems to be parts of town where there 
# are number of these restaurants clustered together
# Query to get the top 50 restaurants with the most violations
query50 = """
    SELECT b.business_id, b.name, b.address, b.latitude, b.longitude, COUNT(*) AS violation_count
    FROM businesses b
    JOIN violations v ON b.business_id = v.business_id
    GROUP BY b.business_id
    ORDER BY violation_count DESC
    LIMIT 50
    """

# Top 50 DF
top_50 = pd.read_sql(query50, connection)

# clean data with missing lon or lat 
cleaned_top_50 = top_50.dropna(subset=['latitude', 'longitude'])

# Display a map of San Francisco
sf_latitude = 37.7749
sf_longitude = -122.4194
sf_map = folium.Map(location=[sf_latitude, sf_longitude], zoom_start=12)

# Add popups for the top 50 restaurants. name, address, and violation count
for index, restaurant in cleaned_top_50.iterrows():
    folium.Marker(
        [restaurant['latitude'], restaurant['longitude']],
        popup=f"{restaurant['name']}Address: {restaurant['address']}Violations: {restaurant['violation_count']}",
        tooltip=restaurant['name']
    ).add_to(sf_map)

# display the map
sf_map

   business_id                     name           address   latitude  \
0         1833  Rogue Ales Public House      673 Union St  37.800254   
1        64176  Dragon River Restaurant   5045 GEARY Blvd  37.780523   
2        32698            Hunan Cafe #2  4450 CABRILLO St  37.773438   
3        71385    Yama Sushi Restaurant  850 Holloway Ave  37.721858   
4        37882               First Cake   5901 GEARY Blvd  37.780087   
6         4621          Louie's Dim Sum  1242 Stockton St  37.797256   
8         3115         Yee's Restaurant    1131 Grant Ave  37.797292   
9        71065        Chutney USA, Inc.      511 Jones St  37.786044   

    longitude  violation_count  
0 -122.410447               49  
1 -122.473580               44  
2 -122.506651               42  
3 -122.462030               40  
4 -122.482596               39  
6 -122.408456               37  
8 -122.406914               36  
9 -122.413054               35  
